<a href="https://colab.research.google.com/github/noname1711/cryptography_theory/blob/main/AES_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycryptodome

In [2]:
import numpy as np
from PIL import Image
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
from google.colab import files

In [3]:
def upload_image():
    uploaded = files.upload()
    for filename in uploaded.keys():
        print(f"Đã tải lên: {filename}")
        return filename

In [4]:
# Hàm đọc ảnh và chuyển sang numpy array
def image_to_array(image_path):
    image = Image.open(image_path)
    return np.array(image), image.mode

In [5]:
# Hàm chuyển numpy array sang ảnh
def array_to_image(array, mode, output_path):
    image = Image.fromarray(array, mode)
    image.save(output_path)

In [19]:
# Mã hóa toàn bộ ảnh
def encrypt_image(image_array, key):
    cipher = AES.new(key, AES.MODE_ECB)  # Sử dụng ECB để đơn giản hóa
    flat_image = image_array.flatten()
    encrypted_data = cipher.encrypt(pad(flat_image.tobytes(), AES.block_size))
    encrypted_array = np.frombuffer(encrypted_data, dtype=np.uint8)

    # Tạo một mảng vuông hợp lệ để lưu thành ảnh
    sqrt_size = int(np.ceil(np.sqrt(len(encrypted_array))))  # Kích thước gần nhất cho ma trận vuông
    padded_array = np.pad(encrypted_array, (0, sqrt_size**2 - len(encrypted_array)), mode='constant', constant_values=0)
    return padded_array.reshape((sqrt_size, sqrt_size)), len(encrypted_array)

In [21]:
# Giải mã toàn bộ ảnh
def decrypt_image(encrypted_array, key, original_shape, data_length):
    cipher = AES.new(key, AES.MODE_ECB)
    flat_encrypted = encrypted_array.flatten()[:data_length]  # Lấy lại dữ liệu thực tế
    decrypted_data = unpad(cipher.decrypt(flat_encrypted.tobytes()), AES.block_size)
    return np.frombuffer(decrypted_data, dtype=np.uint8).reshape(original_shape)

In [23]:
print("1. Mã hóa hình ảnh")
print("2. Giải mã hình ảnh")
choice = input("Nhập 1 để mã hóa hoặc 2 để giải mã: ")

if choice == "1":
    # Mã hóa hình ảnh
    print("Hãy tải lên một hình ảnh:")
    image_path = upload_image()
    image_array, mode = image_to_array(image_path)
    original_shape = image_array.shape

    # Tạo khóa AES ngẫu nhiên
    key = get_random_bytes(16)
    print(f"Khóa AES (vui lòng lưu lại để giải mã): {key.hex()}")

    # Mã hóa dữ liệu ảnh
    encrypted_array, data_length = encrypt_image(image_array, key)

    # Lưu ảnh mã hóa
    encrypted_image_path = "encrypted_" + image_path
    array_to_image(encrypted_array, "L", encrypted_image_path)  # Lưu dưới dạng grayscale
    print(f"Đã mã hóa hình ảnh và lưu tại: {encrypted_image_path}")

    # Lưu thông tin bổ sung để giải mã
    with open("metadata.txt", "w") as meta_file:
        meta_file.write(f"{original_shape}\n{data_length}")

    # Tải xuống file ảnh mã hóa và metadata
    files.download(encrypted_image_path)
    files.download("metadata.txt")

elif choice == "2":
    # Giải mã hình ảnh
    print("Hãy tải lên hình ảnh đã mã hóa:")
    encrypted_image_path = upload_image()
    encrypted_array, mode = image_to_array(encrypted_image_path)

    # Tải metadata
    print("Hãy tải lên tệp metadata:")
    metadata_path = upload_image()
    with open(metadata_path, "r") as meta_file:
        original_shape = eval(meta_file.readline().strip())  # Đọc hình dạng gốc
        data_length = int(meta_file.readline().strip())  # Đọc số lượng dữ liệu thực tế

    # Nhập khóa AES
    key_hex = input("Nhập khóa AES đã sử dụng (dạng hex): ")
    key = bytes.fromhex(key_hex)

    # Giải mã dữ liệu ảnh
    decrypted_array = decrypt_image(encrypted_array, key, original_shape, data_length)

    # Lưu lại ảnh đã giải mã
    decrypted_image_path = "decrypted_" + encrypted_image_path
    array_to_image(decrypted_array, mode, decrypted_image_path)
    print(f"Đã giải mã hình ảnh và lưu tại: {decrypted_image_path}")

    # Tải xuống ảnh đã giải mã
    files.download(decrypted_image_path)

else:
    print("Lựa chọn không hợp lệ. Hãy chạy lại chương trình.")


1. Mã hóa hình ảnh
2. Giải mã hình ảnh
Nhập 1 để mã hóa hoặc 2 để giải mã: 1
Hãy tải lên một hình ảnh:


Saving logo-bach-khoa-ha-noi.jpg to logo-bach-khoa-ha-noi (6).jpg
Đã tải lên: logo-bach-khoa-ha-noi (6).jpg
Khóa AES (vui lòng lưu lại để giải mã): 75fe0e2fe39cf3aeaabe8ecdd74f2cc7
Đã mã hóa hình ảnh và lưu tại: encrypted_logo-bach-khoa-ha-noi (6).jpg


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>